In [1]:
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from tensorflow.keras.layers import Conv2D, Concatenate, MaxPooling2D
from tensorflow.keras.layers import UpSampling2D, Dropout, BatchNormalization
from keras.utils import conv_utils
from tensorflow.keras import backend as K
from keras.layers.core import Activation, SpatialDropout2D

Using TensorFlow backend.


In [18]:
import tensorflow as tf
from tensorflow.keras.applications import Xception as Xception
from tensorflow.keras.applications import DenseNet169 as Densenet
from tensorflow.keras.applications import ResNet50 as Resnet
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import ZeroPadding2D

In [4]:
def conv_block(
    x,
    filters,
    kernel,
    strides=(1,1),
    padding='same',
    activation=None,
    trainable=True,
):
    '''
    X - Input features,
    filters - Output dimension
    kernel - (2,2) Convolution or (3,3) Convolution
    strides - Number of steps taken for consecutive convolution,
    padding = 'same' --> Same across the left,right,top and bottom. or valid --> No padding,
    activation = Activation Layer to be applied.
    '''
    #Input - (224,224,3)
    #Conv2D 
    #- strides --> It divides the input. So it reduce the size of the input. Since we are hoping.
    #- kernel --> 

    x = Conv2D(filters,
            kernel,
            strides,
            padding)(x)  
    x = BatchNormalization(trainable=trainable)(x)  #BatchNorm does not change the dimension of the input. 

    if activation != None:
        if activation == 'relu':
            x = tf.keras.layers.Activation('relu')(x)
        elif activation == 'leakyrelu':
            x = tf.keras.layers.Activation('leakyrelu')(x)
    return x

In [5]:
def residual_block(
    block_input,
    num_filters,):
  
    block_input1 = BatchNormalization()(block_input) #Just adding a precautionary layer, of getting specialized features.
    '''
    We can pass the normalized features to a RELU layer, or we don't it depend on us.
    Let's not pass it. 
    '''
    x = tf.keras.layers.Activation('relu')(block_input)
    print(x.shape)
    x = BatchNormalization()(x)
    x = conv_block(x,
                 filters=num_filters,
                 kernel=(3,3),
                 strides=(1,1),
                 activation='relu')

    x = conv_block(x,
                 num_filters,
                 (3,3),
                 strides=(1,1),
                 activation=None)
    print(x.shape)

    #Assuming the size of the blockinput1 and x is same we will add them.
    #For sanity purpose we will pass the x with a batchnorm.

    x = BatchNormalization()(x)
    return tf.keras.layers.Add()([x,block_input1])

In [6]:
def UXception(input_shape=(None, None, 3),dropout_rate=0.5):

    backbone = Xception(input_shape=input_shape,weights='imagenet',include_top=False)
    input = backbone.input
    start_neurons = 16

    conv4 = backbone.layers[121].output # 32,32,1024
    conv4 = LeakyReLU(alpha=0.1)(conv4) # 32,32,1024
    pool4 = MaxPooling2D((2, 2))(conv4) # 16,16,1024
    pool4 = Dropout(dropout_rate)(pool4) # 16,16,1024
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4) # 16,16,512
    convm = residual_block(convm,start_neurons * 32) # 16,16,512
    convm = residual_block(convm,start_neurons * 32) # 16,16,512
    convm = LeakyReLU(alpha=0.1)(convm) # 16,16,512
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm) # 32,32,256
    uconv4 = concatenate([deconv4, conv4]) # 32,32,1280
    uconv4 = Dropout(dropout_rate)(uconv4) # 32,32,1280
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4) # 32,32,256
    uconv4 = residual_block(uconv4,start_neurons * 16) # 32,32,256
    uconv4 = residual_block(uconv4,start_neurons * 16) # 32,32,256
    uconv4 = LeakyReLU(alpha=0.1)(uconv4) # 32,32,256
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4) #64,64,128
    conv3 = backbone.layers[31].output # 64,64,728
    uconv3 = concatenate([deconv3, conv3])   #64,64,956 
    uconv3 = Dropout(dropout_rate)(uconv3) # 64,64,956
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3) # 64,64,128
    uconv3 = residual_block(uconv3,start_neurons * 8) # 64,64,128
    uconv3 = residual_block(uconv3,start_neurons * 8) # 64,64,128
    uconv3 = LeakyReLU(alpha=0.1)(uconv3) # 64,64,128

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3) # 128,128,64
    conv2 = backbone.layers[21].output # 127,127,256
    conv2 = ZeroPadding2D(((1,0),(1,0)))(conv2) # 128,128,256
    uconv2 = concatenate([deconv2, conv2]) # 128,128,320
        
    uconv2 = Dropout(0.1)(uconv2) # 128,128,320
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2) # 128,128,64
    uconv2 = residual_block(uconv2,start_neurons * 4) # 128,128,64
    uconv2 = residual_block(uconv2,start_neurons * 4) # 128,128,64
    uconv2 = LeakyReLU(alpha=0.1)(uconv2) # 128,128,64
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2) # 256,256,32
    conv1 = backbone.layers[11].output # 253,253,128
    conv1 = ZeroPadding2D(((3,0),(3,0)))(conv1) # 256,256,128
    uconv1 = concatenate([deconv1, conv1]) # 256,256,160
    
    uconv1 = Dropout(0.1)(uconv1) # 256,256,160
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1) # 256,256,32
    uconv1 = residual_block(uconv1,start_neurons * 2) # 256,256,32
    uconv1 = residual_block(uconv1,start_neurons * 2) # 256,256,32
    uconv1 = LeakyReLU(alpha=0.1)(uconv1) # 256,256,32
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   # 512,512,16
    uconv0 = Dropout(dropout_rate)(uconv0) #512,512,16
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0) # 512,512,16
    uconv0 = residual_block(uconv0,start_neurons * 1) # 512,512,16
    uconv0 = residual_block(uconv0,start_neurons * 1) # 512,512,16
    uconv0 = LeakyReLU(alpha=0.1)(uconv0) # 512,512,16
    
    uconv0 = Dropout(dropout_rate/2)(uconv0) # 512,512,16
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    # 512,512,1
    
    model = Model(input, output_layer)
    #model.name = 'u-xception'

    return model

In [7]:
def convolutional_block(input_block,
          start_neurons,
          multiplying_factor,
          kernel=(3,3),
          strides=(1,1),
          padding='same'):
    
    output_filters = start_neurons * multiplying_factor
    
    output_block = Conv2D(output_filters,kernel,activation=None,strides=strides,padding=padding)(input_block)
    output_block = residual_block(output_block,output_filters)
    output_block = residual_block(output_block,output_filters)
    output_block = LeakyReLU(alpha=0.1)(output_block)
    return output_block

In [11]:
def upconv_block(backbone,
                input_block,
                layer_number,
                start_neurons,
                multiplying_factor,
                kernel=(3,3),
                stride=(2,2),
                padding='same',
                padding_kernel=1,
                padding_flag=True,
                dropout_rate=0.1):
    
    print(f'The input block shape is {input_block.shape}')
    
    output_filters = start_neurons * multiplying_factor
    upconv = Conv2DTranspose(output_filters,kernel,strides=stride,padding='same')(input_block)
    layer_output = backbone.layers[layer_number].output
    print(f'The layer output is {layer_output.shape}')
    print(f'The upconv output is shape {upconv.shape}')
    print(f'The padding flag is set to {padding_flag} and padding is {padding_kernel}')
    
    if padding_flag == True:
        print('Inside zero padding')
        layer_output = ZeroPadding2D(((int(padding_kernel),0),(int(padding_kernel),0)))(layer_output)
        print(f'Updated layer_output shape is {layer_output}')
    
    output = concatenate([upconv,layer_output])
    dropout = Dropout(dropout_rate)(output)
    
    return dropout

In [15]:
def Unets(backbone_encoder,input_shape=(None, None, 3),dropout_rate=0.5,start_neurons=16):
    
    if backbone_encoder='Xception':
        
    backbone = Xception(input_shape=input_shape,weights='imagenet',include_top=False)
    input = backbone.input
    
    conv4 = backbone.layers[121].output # 32,32,1024 #16,16,1600
    conv4 = LeakyReLU(alpha=0.1)(conv4) # 32,32,1024 #16,16,1600
    pool4 = MaxPooling2D((2, 2))(conv4) # 16,16,1024  #8,8,1600
    pool4 = Dropout(dropout_rate)(pool4) # 16,16,1024 #8,8,1600
    
    #Middle 
    convm = convolutional_block(pool4,start_neurons=16,multiplying_factor=32) # m64, 8,8,1024
    
#     deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm) # 32,32,256
#     uconv4 = concatenate([deconv4, conv4]) # 32,32,1280
#     uconv4 = Dropout(dropout_rate)(uconv4) # 32,32,1280
    
    
    #Decoder
    uconv4 = upconv_block(backbone,convm,layer_number=121,start_neurons=16,multiplying_factor=16,dropout_rate=0.5,padding_flag=False)
    uconv4 = convolutional_block(uconv4,start_neurons=16,multiplying_factor=16)
    
    uconv3 = upconv_block(backbone,uconv4,layer_number=31,start_neurons=16,multiplying_factor=8,padding_flag=False,dropout_rate=0.5)
    uconv3 = convolutional_block(uconv3,start_neurons=16,multiplying_factor=8)
    
    uconv2 = upconv_block(backbone,uconv3,layer_number=21,start_neurons=16,multiplying_factor=4,padding_kernel=1,padding_flag=True)
    uconv2 = convolutional_block(uconv2,start_neurons=16,multiplying_factor=4)
    
    uconv1 = upconv_block(backbone,uconv2,layer_number=11,start_neurons=16,multiplying_factor=2,padding_kernel=3,padding_flag=True)
    uconv1 = convolutional_block(uconv1,start_neurons=16,multiplying_factor=2)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   # 512,512,16
    uconv0 = Dropout(dropout_rate)(uconv0) #512,512,16
    uconv0 = convolutional_block(uconv0,start_neurons=16,multiplying_factor=1,strides=(1,1))
    
    uconv0 = Dropout(dropout_rate/2)(uconv0) # 512,512,16
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    # 512,512,1
    
    model = Model(input, output_layer)
    
    return model

In [33]:
uxception = Xception(input_shape=(512,512,3),weights='imagenet',include_top=False)

In [41]:
uexception.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 255, 255, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 255, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 255, 255, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [30]:
uexception.layers[121].output.shape

NameError: name 'uexception' is not defined

In [52]:
uexception.layers[121].output.shape

TensorShape([Dimension(None), Dimension(32), Dimension(32), Dimension(1024)])

In [16]:
mm = Unets(input_shape=(512,512,3))

(?, 16, 16, 512)
(?, 16, 16, 512)
(?, 16, 16, 512)
(?, 16, 16, 512)
The input block shape is (?, 16, 16, 512)
The layer output is (?, 32, 32, 1024)
The upconv output is shape (?, 32, 32, 256)
The padding flag is set to False and padding is 1
(?, 32, 32, 256)
(?, 32, 32, 256)
(?, 32, 32, 256)
(?, 32, 32, 256)
The input block shape is (?, 32, 32, 256)
The layer output is (?, 64, 64, 728)
The upconv output is shape (?, 64, 64, 128)
The padding flag is set to False and padding is 1
(?, 64, 64, 128)
(?, 64, 64, 128)
(?, 64, 64, 128)
(?, 64, 64, 128)
The input block shape is (?, 64, 64, 128)
The layer output is (?, 127, 127, 256)
The upconv output is shape (?, 128, 128, 64)
The padding flag is set to True and padding is 1
Inside zero padding
Updated layer_output shape is Tensor("zero_padding2d_3/Pad:0", shape=(?, 128, 128, 256), dtype=float32)
(?, 128, 128, 64)
(?, 128, 128, 64)
(?, 128, 128, 64)
(?, 128, 128, 64)
The input block shape is (?, 128, 128, 64)
The layer output is (?, 253, 253, 1

In [29]:
mm1 = Unets(input_shape=(768,768,3))

(?, 24, 24, 512)
(?, 24, 24, 512)
(?, 24, 24, 512)
(?, 24, 24, 512)
The input block shape is (?, 24, 24, 512)
The layer output is (?, 48, 48, 1024)
The upconv output is shape (?, 48, 48, 256)
The padding flag is set to False and padding is 1
(?, 48, 48, 256)
(?, 48, 48, 256)
(?, 48, 48, 256)
(?, 48, 48, 256)
The input block shape is (?, 48, 48, 256)
The layer output is (?, 96, 96, 728)
The upconv output is shape (?, 96, 96, 128)
The padding flag is set to False and padding is 1
(?, 96, 96, 128)
(?, 96, 96, 128)
(?, 96, 96, 128)
(?, 96, 96, 128)
The input block shape is (?, 96, 96, 128)
The layer output is (?, 191, 191, 256)
The upconv output is shape (?, 192, 192, 64)
The padding flag is set to True and padding is 1
Inside zero padding
Updated layer_output shape is Tensor("zero_padding2d_7/Pad:0", shape=(?, 192, 192, 256), dtype=float32)
(?, 192, 192, 64)
(?, 192, 192, 64)
(?, 192, 192, 64)
(?, 192, 192, 64)
The input block shape is (?, 192, 192, 64)
The layer output is (?, 381, 381, 1

In [17]:
mm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 255, 255, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 255, 255, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 255, 255, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [87]:

import tensorflow as tf

In [88]:
ss = tf.random_normal((1,3,3,3))

In [89]:
ZeroPadding2D(padding=())

TensorShape([Dimension(1), Dimension(3), Dimension(3), Dimension(3)])

In [21]:
m1 = Densenet(input_shape=(512,512,3),weights='imagenet',include_top=False)

51879936/51877672 [==============================] - 257s 5us/step


In [28]:
for idx,layer in enumerate(m1.layers):
    print(f'The index is {idx} , the name of the layer is {layer.name}, the output is {layer.output.shape}')

The index is 0 , the name of the layer is input_4, the output is (?, 512, 512, 3)
The index is 1 , the name of the layer is zero_padding2d_5, the output is (?, 518, 518, 3)
The index is 2 , the name of the layer is conv1/conv, the output is (?, 256, 256, 64)
The index is 3 , the name of the layer is conv1/bn, the output is (?, 256, 256, 64)
The index is 4 , the name of the layer is conv1/relu, the output is (?, 256, 256, 64)
The index is 5 , the name of the layer is zero_padding2d_6, the output is (?, 258, 258, 64)
The index is 6 , the name of the layer is pool1, the output is (?, 128, 128, 64)
The index is 7 , the name of the layer is conv2_block1_0_bn, the output is (?, 128, 128, 64)
The index is 8 , the name of the layer is conv2_block1_0_relu, the output is (?, 128, 128, 64)
The index is 9 , the name of the layer is conv2_block1_1_conv, the output is (?, 128, 128, 128)
The index is 10 , the name of the layer is conv2_block1_1_bn, the output is (?, 128, 128, 128)
The index is 11 , t

In [27]:
m1.layers[1].output.shape

TensorShape([Dimension(None), Dimension(518), Dimension(518), Dimension(3)])

In [34]:
for idx,layer in enumerate(uxception.layers):
    print(f'The index is {idx} , the name of the layer is {layer.name}, the output is {layer.output.shape}')

The index is 0 , the name of the layer is input_8, the output is (?, 512, 512, 3)
The index is 1 , the name of the layer is block1_conv1, the output is (?, 255, 255, 32)
The index is 2 , the name of the layer is block1_conv1_bn, the output is (?, 255, 255, 32)
The index is 3 , the name of the layer is block1_conv1_act, the output is (?, 255, 255, 32)
The index is 4 , the name of the layer is block1_conv2, the output is (?, 253, 253, 64)
The index is 5 , the name of the layer is block1_conv2_bn, the output is (?, 253, 253, 64)
The index is 6 , the name of the layer is block1_conv2_act, the output is (?, 253, 253, 64)
The index is 7 , the name of the layer is block2_sepconv1, the output is (?, 253, 253, 128)
The index is 8 , the name of the layer is block2_sepconv1_bn, the output is (?, 253, 253, 128)
The index is 9 , the name of the layer is block2_sepconv2_act, the output is (?, 253, 253, 128)
The index is 10 , the name of the layer is block2_sepconv2, the output is (?, 253, 253, 128)


In [35]:
1024/16

64.0

In [ ]:
579,361,309,133,45,3

In [38]:
def Unet_dense(input_shape=(None, None, 3),dropout_rate=0.5,start_neurons=16):
    
    backbone = Densenet(input_shape=input_shape,weights='imagenet',include_top=False)
    input = backbone.input
    
    conv4 = backbone.layers[361].output # 32,32,1024 #16,16,1600
    conv4 = LeakyReLU(alpha=0.1)(conv4) # 32,32,1024 #16,16,1600
    pool4 = MaxPooling2D((2, 2))(conv4) # 16,16,1024  #8,8,1600
    pool4 = Dropout(dropout_rate)(pool4) # 16,16,1024 #8,8,1600
    
    #Middle 
    convm = convolutional_block(pool4,start_neurons=16,multiplying_factor=32) # m64, 8,8,1024
    
#     deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm) # 32,32,256
#     uconv4 = concatenate([deconv4, conv4]) # 32,32,1280
#     uconv4 = Dropout(dropout_rate)(uconv4) # 32,32,1280
    
    
    #Decoder
    uconv4 = upconv_block(backbone,convm,layer_number=361,start_neurons=16,multiplying_factor=16,dropout_rate=0.5,padding_flag=False)
    uconv4 = convolutional_block(uconv4,start_neurons=16,multiplying_factor=16)
    
    uconv3 = upconv_block(backbone,uconv4,layer_number=133,start_neurons=16,multiplying_factor=8,padding_flag=False,dropout_rate=0.5)
    uconv3 = convolutional_block(uconv3,start_neurons=16,multiplying_factor=8)
    
    uconv2 = upconv_block(backbone,uconv3,layer_number=45,start_neurons=16,multiplying_factor=4,padding_kernel=1,padding_flag=False)
    uconv2 = convolutional_block(uconv2,start_neurons=16,multiplying_factor=4)
    
    uconv1 = upconv_block(backbone,uconv2,layer_number=3,start_neurons=16,multiplying_factor=2,padding_kernel=3,padding_flag=False)
    uconv1 = convolutional_block(uconv1,start_neurons=16,multiplying_factor=2)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   # 512,512,16
    uconv0 = Dropout(dropout_rate)(uconv0) #512,512,16
    uconv0 = convolutional_block(uconv0,start_neurons=16,multiplying_factor=1,strides=(1,1))
    
    uconv0 = Dropout(dropout_rate/2)(uconv0) # 512,512,16
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv0)    # 512,512,1
    
    model = Model(input, output_layer)
    
    return model

In [42]:
den = Unet_dense(input_shape=(768,768,3))

(?, 24, 24, 512)
(?, 24, 24, 512)
(?, 24, 24, 512)
(?, 24, 24, 512)
The input block shape is (?, 24, 24, 512)
The layer output is (?, 48, 48, 128)
The upconv output is shape (?, 48, 48, 256)
The padding flag is set to False and padding is 1
(?, 48, 48, 256)
(?, 48, 48, 256)
(?, 48, 48, 256)
(?, 48, 48, 256)
The input block shape is (?, 48, 48, 256)
The layer output is (?, 96, 96, 128)
The upconv output is shape (?, 96, 96, 128)
The padding flag is set to False and padding is 1
(?, 96, 96, 128)
(?, 96, 96, 128)
(?, 96, 96, 128)
(?, 96, 96, 128)
The input block shape is (?, 96, 96, 128)
The layer output is (?, 192, 192, 128)
The upconv output is shape (?, 192, 192, 64)
The padding flag is set to False and padding is 1
(?, 192, 192, 64)
(?, 192, 192, 64)
(?, 192, 192, 64)
(?, 192, 192, 64)
The input block shape is (?, 192, 192, 64)
The layer output is (?, 384, 384, 64)
The upconv output is shape (?, 384, 384, 32)
The padding flag is set to False and padding is 3
(?, 384, 384, 32)
(?, 384,

In [43]:
den.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 768, 768, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_15 (ZeroPadding2 (None, 774, 774, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 384, 384, 64) 9408        zero_padding2d_15[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 384, 384, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv4_block28_1_bn (BatchNormal (None, 48, 48, 128)  512         conv4_block28_1_conv[0][0]       
__________________________________________________________________________________________________
conv4_block28_1_relu (Activatio (None, 48, 48, 128)  0           conv4_block28_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block28_2_conv (Conv2D)   (None, 48, 48, 32)   36864       conv4_block28_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block28_concat (Concatena (None, 48, 48, 1152) 0           conv4_block27_concat[0][0]       
                                                                 conv4_block28_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block29_0_bn (BatchNormal (None, 48, 48, 1152) 4608        conv4_block28_concat[0][0]       
__________